**Managed Vs External Table in PySpark:**

In PySpark (and Spark SQL in general), the terms managed and external tables refer to how Spark handles the data storage and metadata for those tables in a data warehouse or distributed file system (such as HDFS or S3). Here’s a breakdown of the differences:

**1. Managed Table**

A managed table (also called an internal table) is a table where Spark manages both the metadata and the data.

-   **Storage Location:** Spark stores both the data and metadata in the default location defined by the Spark warehouse. Typically, this is a directory like /user/hive/warehouse in HDFS or a similar path in other storage systems (like S3). You don't need to specify the location explicitly; Spark handles it.

-   **Data Management:** Spark owns the lifecycle of both the data and the table. If you drop the table, Spark will delete both the metadata and the data associated with that table.

-   **Use Case:** Managed tables are typically used when Spark is responsible for managing the lifecycle of the data and metadata. This is useful when you're working with data that will be used and processed entirely within the Spark ecosystem.

**Example of Creating a Managed Table:**

```
spark.sql("CREATE TABLE managed_table (id INT, name STRING) USING parquet")
```

If you drop the table:

```
spark.sql("DROP TABLE managed_table")
```

This will delete the `table and its data` from the storage location.

**2. External Table**

An external table refers to a table where Spark manages only the metadata, but not the actual data. The data for an external table resides outside Spark's managed directory, typically in an external storage system like HDFS, S3, or another database.

-   **Storage Location:** The data is stored externally (you specify the path explicitly, such as a directory on HDFS or S3). Spark only manages the table metadata, not the actual data files.

-   **Data Management:** Spark does not manage the data in external tables. If you drop an external table, only the metadata is removed, not the actual data files.

-   **Use Case:** External tables are useful when the data resides in a location managed outside Spark (e.g., HDFS, S3, or a relational database). This allows other systems to access the same data, and Spark can work with it without interfering with the actual data storage.

**Example of Creating an External Table:**

```
spark.sql("""
CREATE EXTERNAL TABLE external_table (id INT, name STRING)
STORED AS parquet
LOCATION 's3://my-bucket/external_data/'
""")
```

If you drop the table:

```
spark.sql("DROP TABLE external_table")
```

This will `only delete the metadata`, and the data files in s3://my-bucket/external_data/ will remain intact.

**Summary of Key Differences**


| Feature                  | Managed Table                          | External Table                       |
|--------------------------|----------------------------------------|--------------------------------------|
| **Data Management**       | Spark manages both data and metadata.  | Spark manages metadata only. Data remains in the specified external location. |
| **Storage Location**      | Spark defines and manages the storage location. Typically in the default warehouse directory. | You define the storage location (e.g., HDFS, S3). |
| **Data Deletion on Drop** | Both data and metadata are deleted.    | Only metadata is deleted; data remains. |
| **Use Case**              | When Spark is responsible for both the data and metadata management. | When the data is managed externally and Spark only needs to manage the metadata. |

**When to Use Which:**

-   **Managed Tables:** Use them when you want Spark to have full control over both the table metadata and the data. This is ideal for temporary or intermediate data that you don’t want to manage outside Spark.

-   **External Tables:** Use them when you want to reference data that already exists outside Spark’s control, such as in HDFS, S3, or another system, and you don't want Spark to delete the actual data when dropping the table.

In [1]:
import findspark
findspark.init
import getpass
from pyspark.sql import SparkSession

username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config("spark.sql.catalogImplementation", "hive"). \
    config("spark.sql.warehouse.dir",f"/Users/{username}/Documents/data/warehouse"). \
    enableHiveSupport(). \
    master("local"). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 19:24:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("use retail")

24/12/25 19:24:24 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/25 19:24:24 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/12/25 19:24:25 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/12/25 19:24:25 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore sugumarsrinivasan@192.168.31.195
24/12/25 19:24:25 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [3]:
spark.sql("create table if not exists orders_ext (order_id integer, order_date string, customer_id integer, order_status string) using csv location '/Users/sugumarsrinivasan/Documents/data/orders'")

24/12/25 19:26:56 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`retail`.`orders_ext` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/12/25 19:26:56 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/12/25 19:26:56 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/12/25 19:26:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/12/25 19:26:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [4]:
spark.sql("show tables").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|   retail|orders_ext|      false|
+---------+----------+-----------+



In [17]:
spark.sql("select * from spark_catalog.retail.orders_ext limit 5").show(truncate=False)

+--------+---------------------+-----------+------------+
|order_id|order_date           |customer_id|order_status|
+--------+---------------------+-----------+------------+
|1       |2013-07-27 00:00:00.0|30265      |CLOSED      |
|2       |2013-11-25 00:00:00.0|20386      |CLOSED      |
|3       |2014-01-21 00:00:00.0|15768      |COMPLETE    |
|4       |2014-07-04 00:00:00.0|27181      |PROCESSING  |
|5       |2014-03-08 00:00:00.0|12448      |COMPLETE    |
+--------+---------------------+-----------+------------+



In [6]:
spark.sql("describe formatted orders_ext").show(truncate=False)

+----------------------------+---------------------------------------------------------+-------+
|col_name                    |data_type                                                |comment|
+----------------------------+---------------------------------------------------------+-------+
|order_id                    |int                                                      |NULL   |
|order_date                  |string                                                   |NULL   |
|customer_id                 |int                                                      |NULL   |
|order_status                |string                                                   |NULL   |
|                            |                                                         |       |
|# Detailed Table Information|                                                         |       |
|Catalog                     |spark_catalog                                            |       |
|Database                    |

In [ ]:
spark.sql("truncate table spark_catalog.retail.orders_ext")

# output:
# AnalysisException: Operation not allowed: TRUNCATE TABLE on external tables: `spark_catalog`.`retail`.`orders_ext`.

Data Manipulation Language (DML) operations like INSERT, UPDATE, DELETE, and SELECT can be performed on external tables in PySpark, but there are some important distinctions and limitations to be aware of. Here's a breakdown:

**1. SELECT**

-   **Works on External Tables:** The SELECT operation works as expected on external tables because it queries the data stored externally (e.g., in HDFS, S3, etc.) based on the table schema (metadata).

**Example:**

```
spark.sql("SELECT * FROM external_table")
```

**2. INSERT**

-   **Works on External Tables:** You can perform INSERT operations to add data into an external table. This will write data into the location defined for the external table (e.g., a directory in HDFS or S3).

**Example:**

```
spark.sql("INSERT INTO external_table VALUES (1, 'John')")
```

However, keep in mind that Spark will append the data to the external location, so it doesn't manage partitions or data consistency the same way as managed tables.

**3. UPDATE**

-   **Limited Support for External Tables:**

    -   **Not directly supported in Spark:** Spark does not support UPDATE operations natively on external tables because Spark writes data in a distributed and immutable way. The data in external storage systems like HDFS or S3 is not updated in-place; instead, Spark typically rewrites data during operations like INSERT or MERGE.
    -   For UPDATE-like functionality, you can use Delta Lake (if using Delta format) or use a workaround with overwriting data.

**Workaround using DataFrame operations (not ideal):**

```
df = spark.read.parquet("s3://my-bucket/external_table/")

# Perform transformations (e.g., updating a value)
updated_df = df.withColumn("name", F.when(df.id == 1, "Updated Name").otherwise(df.name))

# Overwrite the external table data
updated_df.write.mode("overwrite").parquet("s3://my-bucket/external_table/")
```

-   **Delta Lake Support:** If you are using Delta Lake, the UPDATE operation is supported on external tables as it provides ACID transactions.

**Example with Delta:**

```
delta_table = DeltaTable.forPath(spark, "s3://my-bucket/external_table/")
delta_table.update(condition = "id = 1", set = {"name": "'Updated Name'"})
```

**4. DELETE**

-   **Limited Support for External Tables:**
    -   Like UPDATE, the DELETE operation is not natively supported in Spark on external tables because Spark doesn't handle row-level deletions in the same way a traditional RDBMS might.
    -   Workaround using DataFrame operations: You can filter out rows and rewrite the data (similar to how UPDATE is handled).

**Workaround using DataFrame operations:**
```
df = spark.read.parquet("s3://my-bucket/external_table/")

# Filter out rows you want to delete
df_filtered = df.filter(df.id != 1)

# Overwrite the external table data with the remaining rows
df_filtered.write.mode("overwrite").parquet("s3://my-bucket/external_table/")
```

**Delta Lake Support:** Delta Lake supports DELETE operations on external tables with ACID transactions.

**Example with Delta:**

```
delta_table = DeltaTable.forPath(spark, "s3://my-bucket/external_table/")
delta_table.delete("id = 1")
```


**Summary of DML Operations on External Tables**

| DML Operation | Managed Tables | External Tables (Non-Delta) | External Tables (Delta Lake) |
|---------------|----------------|-----------------------------|------------------------------|
| **SELECT**    | Yes            | Yes                         | Yes                          |
| **INSERT**    | Yes            | Yes                         | Yes                          |
| **UPDATE**    | Yes            | No (unless using overwrite) | Yes (with Delta Lake)        |
| **DELETE**    | Yes            | No (unless using overwrite) | Yes (with Delta Lake)        |


**Key Points:**

-   SELECT and INSERT are fully supported on external tables.

-   UPDATE and DELETE are not natively supported on external tables unless using Delta Lake for ACID transactions.

-   For non-Delta external tables, you can workaround UPDATE and DELETE by reading the data, modifying it, and overwriting the external location.

**Recommendation:**

-   If you need advanced support for DML operations (especially UPDATE and DELETE), consider using Delta Lake (which provides ACID transactions and better handling of these operations).

In [ ]:
spark.sql("insert into spark_catalog.retail.orders_ext values(105, '2013-07-27 00:00:00.0', 5555, 'CLOSED')")

DataFrame[]